# Test Environment
data\
    talent.xlsx\
    waffen.xlsx\
    zauber.xlsx

einheit\
    held1.xlsx\
    held1_inventar.xlsx

In [13]:
# Modul um Würfel zu simulieren
import random
random.seed(1321345589)

# Modul zum vereinfachten Auslesen und Bearbeiten von Datentabellen
import pandas as pd

# Dies und Das
import regex as re

In [14]:
def w20():
    """
    Simple Simulation eines W20 Würfels.
    Benötigt keinen Input und gibt immer einen Wert zwischen 1 und 20 zurück.
    """

    return random.randint(1, 20)


def w6(anzahl: int = 1):
    """
    Simple Simulation eines W6 Würfels.
    Benötigt keinen Input und gibt immer einen Wert zwischen 1 und 6 zurück.
    """

    assert anzahl > 0

    wuerfe = [random.randint(1, 6) for _ in range(anzahl)]

    return sum(wuerfe)


def datei_zu_dataframe(path, index_column=0):
    """
    
    """

    dataframe = pd.read_excel(path, index_col=index_column)
    dataframe.dropna(inplace=True)

    if "Eigenschaftsprobe" in dataframe.columns:
        dataframe["Eigenschaftsprobe"] = dataframe.Eigenschaftsprobe.apply(lambda x: x.split("/"))

    return dataframe


def datei_zu_dict(path):
    """
    Liest excel Tabelle mit Informationen zum Helden ein, damit diese als
    Klassen-Attribute übergeben werden können.
    
    : name : Dateiname bzw. Name des Helden, Pfad in /einheit/name
    : return : dict
    """

    frame = datei_zu_dataframe(path)
    framedict = frame.T.to_dict(orient="records")[0]

    return framedict


def zeile_zu_dict(frame, zeile):
    """
    
    """

    pass

In [15]:
# Einlesen der Datentabelle zu den Fertigkeiten
dsa5_talente = datei_zu_dataframe("data/dsa5_talente.xlsx", index_column=1)

dsa5_waffen = datei_zu_dataframe("data/waffen.xlsx")
dsa5_ruestungen = datei_zu_dataframe("data/ruestungen.xlsx")
dsa5_zauber = datei_zu_dataframe("data/zauber.xlsx")

dsa5_waffen

,Art,TP,Bonus,L+S,KK,AT-Mod,PA-Mod,Reichweite,Vorteil,Nachteil
Name,,,,,,,,,,


## Glossar

In [16]:
# Kurzes Glossar durch die ganzen Abkürzungen (besser für die Textausgabe)
# Benutzt ein dictionary, also glossar["MU"] ergibt als Ausgabe "Mut"
glossar = datei_zu_dict("data/glossar.xlsx")

# Das Glossar wird verdoppelt, wobei die Dopplung Keys und Values miteinander vertauscht.
# Also das dict glossar ist nach dem "glossar[key] = value"-Prinzip aufgebaut, dies wird
# mit der dict comprehension (also {val: key for ...}) gedreht. Das daraus entstehende
# neue dict wird dann dem "alten" Glossar beigefügr (der Operator | verbindet zwi dicts.
# Damit ist jetzt auch glossar["Mut"] = "MU" gewährleistet.
glossar = glossar | {val: key for key, val in glossar.items()}

In [17]:
data_margot = datei_zu_dict("data/held_margot.xlsx")
data_margot

{'name': 'Margot',
 'alter': 31,
 'rasse': 'Mensch',
 'heldentyp': 'Soldatin',
 'MU': 8,
 'KL': 8,
 'IN': 8,
 'CH': 8,
 'FF': 8,
 'GE': 8,
 'KO': 8,
 'KK': 8,
 'LeP': 21,
 'AsP': 4,
 'Int': 12,
 'AP': 0}

In [18]:
class Einheit():
    def __init__(self, filepath):
        if filepath is not None:
            file = datei_zu_dict(filepath)

            for key, value in file.items():
                setattr(self, key, value)
        
        self.haupthand = "Bastardschwert"
    
    
    def attacke(self, other, aktion):
        pass


    def parade(self, other):
        pass


    def eigenschaftsprobe(self, probe: str, bias: int=0):
        wurf = w20()

        if wurf > getattr(self, glossar[probe]) + bias:
            print(f"Eigenschaftsprobe misslungen! Der nötige {probe}-Wert von {getattr(self, glossar[probe])} wurde durch {wurf} nicht unterworfen!")
        
        else:
            print(f"Eigenschaftsprobe geschafft! Der nötige {probe}-Wert von {getattr(self, glossar[probe])} wurde durch {wurf} unterworfen!")
    

    def fertigkeitsprobe(self, probe: str, bias: int=0):
        """
        
        """

        ausgleich = random.randint(3, 7)
        print(f'Starte Fertigkeitsprobe {probe} auf {dsa5_talente["Eigenschaftsprobe"][probe]}, Ausgleich: {ausgleich}')

        for i, eigenschaft in enumerate(dsa5_talente["Eigenschaftsprobe"][probe]):
            wurf = w20()

            ziel = getattr(self, eigenschaft) + bias
            wurfdifferenz = ziel - wurf

            if wurfdifferenz < 0:
                ausgleich += wurfdifferenz
            
                if ausgleich < 0:
                    print(f'\nFertigkeitsprobe misslungen! {glossar[eigenschaft]} wurde mit {wurf} nicht unterworfen (Ziel: {getattr(self, eigenschaft)} mit Bias {bias}), Ausgleich wurde aufgebraucht ({ausgleich}).')

                    return
                
                else:
                    print(f'Teilprobe fehlgeschlagen! {glossar[eigenschaft]} wurde mit {wurf} nicht unterworfen (Ziel: {getattr(self, eigenschaft)} mit Bias {bias}), verbleibender Ausgleich: {ausgleich}.')
            
            else:
                print(f'Teilprobe geglückt! {glossar[eigenschaft]} wurde mit {wurf} unterworfen (Ziel: {getattr(self, eigenschaft)} mit Bias {bias}), verbleibender Ausgleich: ({ausgleich}).')
        
        print(f'\nFertigkeitsprobe geglückt! Übriger Ausgleich bei {ausgleich}.')

In [19]:
margot = Einheit("data/held_margot.xlsx")

margot.fertigkeitsprobe("Tierkunde")

Starte Fertigkeitsprobe Tierkunde auf ['MU', 'MU', 'CH'], Ausgleich: 7

Fertigkeitsprobe misslungen! Mut wurde mit 19 nicht unterworfen (Ziel: 8 mit Bias 0), Ausgleich wurde aufgebraucht (-4).
